# **Amazon Model - SBERT**




In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
embed = SentenceTransformer('stsb-bert-base')

In [ ]:
get_ipython().system('pip install dgl')

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import heapq

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report

import pandas as pd

import dgl
from dgl.data import DGLDataset
import torch

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

## Skippable

In [ ]:
PATH = '/content/drive/MyDrive/kaggle/amazon_review_full_csv/train.csv'

In [ ]:
df = pd.read_csv(PATH, sep = ",")
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.columns = ['rating', 'title', 'review']
df.head()

In [ ]:
df1 = df.copy()

# Change this for accessing more data
data_len = 10000

for i in range(data_len):

    if df1.rating[i] < 3:
        df1.rating[i] = 1
    else:
        df1.rating[i] = 0

In [ ]:
df1.head()

In [ ]:
tweets = []
labels = list(df1.rating[:data_len])

for i in range(data_len):
    
    tweets.append(embed.encode([df1.review[i]]))

    if i % 100 == 1:
        print(i, 'tweets embedded')

In [ ]:
tweets = np.stack(tweets).reshape((len(tweets), 768))
df1 = pd.DataFrame(tweets)
df1.head(3)

In [ ]:
# Cosine similarity between the tweet vectors

cos_res_temp = cosine_similarity(df1, df1)
print('cos_res_temp:\n', cos_res_temp)
print(cos_res_temp.shape)

In [ ]:
# To ensure that each node has attleast 3 connections

for i in range(len(cos_res_temp)):
    ind = heapq.nlargest(3, range(len(cos_res_temp[i])), key=lambda x: cos_res_temp[i][x])
    for j in ind:
        cos_res_temp[i][j] = 1


In [ ]:
cos_res_temp

In [ ]:
# Diagonal of adjacency matrix = Degree of node
cos_res = (cos_res_temp - 0.25).round() - np.identity(len(cos_res_temp))
degree = np.sum(cos_res, axis = 1)

print('Max:', max(degree), '\nMin:', min(degree), '\nAvg:', sum(degree)/len(degree))

In [ ]:
cos_res

In [ ]:
cnt=0
start_indx  = []
end_idx = []
weights = []

for i in range(len(tweets)):
    for j in range(len(tweets)):
        if cos_res[i][j]==1 :
            cnt+=1
            start_indx.append(i)
            end_idx.append(j)
            # weights.append(cos_res_temp[i][j])

In [ ]:
edge_dataFrame = pd.DataFrame({'target':start_indx, 'source':end_idx})
#edges_data = pd.DataFrame(weights)
edge_dataFrame.head()

In [ ]:
class TweetDataSet(DGLDataset):
    def __init__(self):
        super().__init__(name='tweet_data')

    def process(self):
        
        node_features = torch.from_numpy(df1.values).float()
        node_labels = torch.from_numpy(np.stack(labels))
        
        edges_src = torch.from_numpy(edge_dataFrame['source'].to_numpy())
        edges_dst = torch.from_numpy(edge_dataFrame['target'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=df1.shape[0])
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels
        
        n_nodes = df1.shape[0]
        n_train = int(n_nodes * 0.8)
        n_val = int(n_nodes * 0.1)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = TweetDataSet()
graph = dataset[0]

print(graph)

In [ ]:
# Final Model - Graph Attention Network

class GNN_final(nn.Module):

    def __init__(self, in_feats, out_feats):
        super().__init__()
        
        h1 = 256 * 1
        h2 = 128 * 1
        h3 = 32 * 1

        self.gat1 = dglnn.conv.GATConv(in_feats = in_feats, out_feats = h1, num_heads = 8, attn_drop = 0.2, residual  = True, feat_drop = 0.2)
        self.gat2 = dglnn.conv.GATConv(in_feats = h1, out_feats = h2, num_heads = 12, attn_drop = 0.2, residual  = True, feat_drop = 0.2)
        self.gat3 = dglnn.conv.GATConv(in_feats = h2, out_feats = h3, num_heads = 8, attn_drop = 0.2, residual  = True, feat_drop = 0.2)
        
        self.dense1 = torch.nn.Linear(in_features = h1 + h2 + h3, out_features = h3)
        self.dense2 = torch.nn.Linear(in_features = h3, out_features = 2)
        
        self.dense3 = torch.nn.Linear(in_features = h1 + h2 + h3, out_features = h3)
        self.dense4 = torch.nn.Linear(in_features = h3, out_features = 3)

    def forward(self, graph, inputs):

        h1 = self.gat1(graph, inputs)
        h1 = F.relu(h1.mean(dim=1))

        h2 = self.gat2(graph, h1)
        h2 = F.relu(h2.mean(dim=1))
        
        h3 = self.gat3(graph, h2)
        h3 = F.relu(h3.mean(dim=1))

        h4 = self.dense1(torch.hstack((h1, h2, h3)))
        h5 = self.dense2(F.relu(h4))

        return torch.sigmoid(h5)

In [ ]:
# Model 1 - Graph Attention Network

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

class GNN1(nn.Module):

    def __init__(self, in_feats, out_feats):
        super().__init__()
        
        h1 = 256 * 1
        h2 = 128 * 1
        h3 = 32 * 1

        self.gat1 = dglnn.conv.GATConv(in_feats = in_feats, out_feats = h1, num_heads = 4)
        self.gat2 = dglnn.conv.GATConv(in_feats = h1, out_feats = h2, num_heads = 4)
        self.gat3 = dglnn.conv.GATConv(in_feats = h2, out_feats = h3, num_heads = 4)
        
        self.dense1 = torch.nn.Linear(in_features = h1 + h2 + h3, out_features = h3)
        self.dense2 = torch.nn.Linear(in_features = h3, out_features = 2)
        
    def forward(self, graph, inputs):

        h1 = self.gat1(graph, inputs)
        h1 = F.relu(h1.mean(dim=1))

        h2 = self.gat2(graph, h1)
        h2 = F.relu(h2.mean(dim=1))
        
        h3 = self.gat3(graph, h2)
        h3 = F.relu(h3.mean(dim=1))

        h4 = self.dense1(torch.hstack((h1, h2, h3)))
        h4 = F.relu(h4)

        h5 = self.dense2(h4)

        return torch.sigmoid(h5)

In [ ]:
# Model 2 - SAGE network

class GNN2(nn.Module):
    
    def __init__(self, in_feats, out_feats):
        super().__init__()

        h1 = 256
        h2 = 128
        h3 = 32

        self.conv1 = dglnn.SAGEConv(in_feats = in_feats, out_feats=h1, aggregator_type = 'pool', activation = F.relu, bias = True)
        self.conv2 = dglnn.SAGEConv(in_feats = h1, out_feats=h2, aggregator_type = 'pool', activation = F.relu, bias = True)
        self.conv3 = dglnn.SAGEConv(in_feats = h2, out_feats=h3, aggregator_type = 'pool', activation = F.relu, bias = True)
        
        self.dense1 = torch.nn.Linear(in_features = h1 + h2 + h3, out_features = h3)
        self.dense2 = torch.nn.Linear(in_features = h3, out_features = 2)
    
    def forward(self, graph, inputs):

        h1 = self.conv1(graph, inputs)
        h2 = self.conv2(graph, h1)
        h3 = self.conv3(graph, h2)
        
        h4 = self.dense1(torch.hstack((h1, h2, h3)))
        h4 = F.relu(h4)

        h5 = self.dense2(h4)

        return torch.sigmoid(h5)

In [ ]:
# Model 3 - Graph Convolution Network

import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

class GNN3(nn.Module):

    def __init__(self, in_feats, out_feats):
        super().__init__()

        h1 = 256
        h2 = 128
        h3 = 32

        self.gc1 = dglnn.conv.GraphConv(in_feats = in_feats, out_feats = h1, activation  = F.relu)
        self.gc2 = dglnn.conv.GraphConv(in_feats = h1, out_feats = h2, activation  = F.relu)
        self.gc3 = dglnn.conv.GraphConv(in_feats = h2, out_feats = h3, activation  = F.relu)
         
        self.dense1 = torch.nn.Linear(in_features = h1 + h2 + h3, out_features = h3)
        self.dense2 = torch.nn.Linear(in_features = h3, out_features = 2)

    def forward(self, graph, inputs):

        h1 = self.gc1(graph, inputs)
        h2 = self.gc2(graph, h1)        
        h3 = self.gc3(graph, h2)

        h4 = self.dense1(torch.hstack((h1, h2, h3)))
        h4 = F.relu(h4)

        h5 = self.dense2(h4)

        return torch.sigmoid(h5)

In [ ]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label'].long()
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [ ]:
def train(g, model):

    t_acc = []
    v_acc = []
    t_loss = []

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0001)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
            
        t_acc.append(train_acc)
        v_acc.append(val_acc)
        t_loss.append(loss) 

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tweets/BERT/BERT_Amazon_Best.pt')

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc: {:.3f} val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, train_acc, val_acc, best_val_acc, test_acc, best_test_acc))
        
    return t_acc, v_acc, t_loss 

model = GNN_final(in_feats = n_features, out_feats = n_labels)
t_acc, v_acc, t_loss = train(graph, model)

torch.save(model.state_dict(), '/content/drive/MyDrive/Tweets/BERT/BERT_Amazon_Final.pt')

In [ ]:
print(model)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize= (13, 7))
plt.plot(t_acc)
plt.plot(v_acc)

In [ ]:
plt.figure(figsize= (13, 7))
plt.plot(t_loss)

In [ ]:
best_model = GNN_final(in_feats = n_features, out_feats = n_labels)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/Tweets/BERT/BERT_Amazon_Best.pt'))

In [ ]:
def evaluate(model, graph, features, labels, mask):
  model.eval()
  with torch.no_grad():
      logits = model(graph, features)
      logits = logits[test_mask]
      labels = labels[test_mask]
      k, indices = torch.max(logits, dim=1)

      preds = []
      
      for i in logits:
        preds.append(np.argmax(i))

      print(classification_report(labels, preds, labels=[0, 1]))
      return preds,k

print('Final Model:')
preds, k = evaluate(model, graph, node_features, node_labels, test_mask)

print('---------------------------')

print('Best model:')
preds, k = evaluate(best_model, graph, node_features, node_labels, test_mask)

In [ ]:
print( '# zeros :', len(labels) - sum(labels), '\n# ones :', sum(labels))